# Implementation of *innate training* (pre-training)

1. Overview
2. Purpose
3. Preparation
4. Implementation
    - Base module
    - Emulator
5. Demonstrations

## 1. Overview
Let's implement an online training scheme using an echo state network (ESN) called *innate training* proposed by [R. Laje et al. (2013)](https://www.nature.com/articles/nn.3405).
Innate training offers a two-step training process *i. e.,* **pre-training** and **post-training**.

In the pre-training process, innate training tunes the internal connection of ESNs to allow them to reproducibly generate high dimensional chaotic dynamics called *innate trajectory* according to a specific input time series, which is achieved by incorporating [FORCE learning [D. Sussillo et al., 2009]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2756108/).

After the pre-training process, the high-dimensional complicated trajectory would have rich expressiveness to design arbitrary outputs. In the post-training process, an external linear readout is tuned to output a desired trajectory.

In this notebook, let's first implement the pre-training process of innate training.

## 2. Purpose
We are implementing an online training setup with innate training and replicating the paper's results by exploiting high-dimensional chaotic time series. 

## 3. Preparation
Just run the following importing scripts (press `shift+Enter` to run each cell).
You will see an authentification popup, then please accept it.

In [ ]:
import os
import sys
import json
import joblib
import numpy as np
from collections import defaultdict

In [ ]:
if "google.colab" in sys.modules:
    from google.colab import drive  # Connecting to google drive
    drive.mount("/content/gdrive")
    %cd /content/gdrive/My Drive/sandbox/rc_tutorial/closed_loop_control/
    # NOTE: change it to your own path
    # e.g., %cd /content/gdrive/My Drive/[PATH_TO_NOTEBOOK]
    from utils.magic import colab_script  # save and load cells
    exec(colab_script)
else:
    sys.path.append(".")
    from utils.magic import run_and_save, load_and_run  # save and load cells

from utils.tqdm import trange, tqdm  # progress bar
from utils.reservoir import Module, ESN  # class for echo state network
from utils.interpolate import interp1d  # interpolation
from utils.style_config import plt
from views.innate_training import *  # visualization wrapper
from tests.innate_training import *  # check function

# 4. Implementation

## 4.1 Base module
Let's implement `InnateESN` by overriding `ESN` to support the innate training algorithm.

By considering each pre-synaptic connection of each neuron as a linear closed-loop, the innate training optimizes them as with the feedback loop training in the FORCE learning scheme. 
The following equations show the weight update rule based on FORCE learning for $i$-th pre-synaptic connection ($1\leq i \leq N$);
$$
\def\bm#1{{\boldsymbol #1}}
\begin{align*}
\bm{k}^{i} &= P^{i} \bm{x}^{i} \\
g^{i} &= \frac{1}{1+{\bm{x}^{i}}^T\bm{k}^{i}} \\
\Delta P^{i} &= g^{i}{\bm{k}}^{i}{\bm{k}^{i}}^{T} \\ 
\Delta W^\text{net}_{i} &= g^{i} (\hat{\bm{x}}^{i} - \bm{x}^{i}) \bm{k}^{i} \\
P^{i} &\leftarrow P^{i} - \Delta P^{i} \\
W^\text{net}_{i}  &\leftarrow W^\text{net}_{i}  + \Delta W^\text{net}_{i},
\end{align*}
$$
where $P^{i} \in \mathbb{R}^{N^{i}\times N^{i}}$ is an auxiliary positive-definite matrix initialized as ${I}/{\alpha}$, and $\hat{\bm{x}}$ the innate trajectory which will be generated later.
You will be aware that this process is almost the same as the previous one, and thereby you can reuse the `rls_update` you have implemented. Note that $N^{i}$ takes a smaller value than $N$ since the ESN has sparse connectivity (`sparse < 1.0`, $p<1.0$), drastically reducing the computational complexity ($O(p^2 N^3)$ for each iteration).

Please fill blanks in `InnateESN.__init__` and `InnateESN.train` to implement the algorithm. 

Specification of `InnateESN.train`
- Arguments
    - `x_target` (`np.ndarray`): State on innate trajectory $\hat{\bm{x}}(t)$ ($\in \mathbb{R}^{\cdots \times N}$).
    - `x_now` (`np.ndarray`): Current state $\bm{x}(t)$ ($\in \mathbb{R}^{\cdots \times N}$).
- Return
    - `np.ndarray`: Difference of matrix ($\Delta W$).
- Operations
    - Update `InnateESN.P` ($P^i$) with `rls_update`, which are initialized $I/\alpha$ in `__init__`.
    - Update `InnateESN.w_net` with `rls_update`.

<details>
<summary>
Hints
</summary>

- `InnateESN.w_net` is a sparse matrix.
- Read [`np.nonzero`'s documantation](https://numpy.org/doc/stable/reference/generated/numpy.nonzero.html) and understand the variables in `__init__`.

In [ ]:
%load_and_run ./answer/rls_update.py  
# NOTE: Please replace it with your implementation if the file does not exist.

In [ ]:
%%run_and_save ./answer/innate_esn.py

class InnateESN(ESN):
    def __init__(self, *args, alpha=1.0, **kwargs):
        '''
        Tunable echo state network [Laje, R., & Buonomano, D. V. (2013). Nature neuroscience, 16(7), 925-933.]

        Args:
            alpha (float, optional): regularization parameter for RLS algorithm. Defaults to 1.0.
        '''
        super(InnateESN, self).__init__(*args, **kwargs)
        self.alpha = alpha
        self.w_pre = {}
        self.P = {}
        for post in range(self.dim):
            non_zeros = self.w_net[post].nonzero()[0]
            self.w_pre[post] = non_zeros
            self.P[post] = ... # TODO initialize P

    def train(self, x_target, x_now=None, neuron_list=None):
        '''
        Update the internal weight by RLS algorithm

        Args:
            x_target (np.ndarray): State(s) on an inante trajectory.
            x_now (np.ndarray, optional): Current state(s). Defaults to None (use self.x).
            neuron_list (list, slice, optional): Tuned nodes. Defaults to None (train all nodes).
        '''
        if x_now is None:
            x_now = np.asarray(self.x)
        if neuron_list is None:
            neuron_list = range(self.dim)
        for xt, xn in zip(x_target.reshape(-1, self.dim), x_now.reshape(-1, self.dim)):
            es = xt[neuron_list] - xn[neuron_list]
            for node_id, e in zip(neuron_list, es):
                x = ... # TODO use self.w_pre. (big hint: x = xn[...])
                P = ... # TODO self.P
                g, k, P_new = rls_update(P, x)
                dw = ... # TODO
                self.P[node_id] = P_new
                self.w_net[node_id, self.w_pre[node_id]] += dw
        return self.w_net

    def to_pickle(self, file_name):
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        P, w_pre = self.P, self.w_pre
        self.P, self.w_pre = {}, {}
        with open(file_name, mode="wb") as f:
            joblib.dump(self, f, compress=True)
        self.P, self.w_pre = P, w_pre

    @staticmethod
    def read_pickle(file_name):
        with open(file_name, mode="rb") as f:
            module = joblib.load(f)
        return module

In [ ]:
test_innate_esn(InnateESN)

## 4.2 Emulator

Next, let's implement a function controlling the pre-training of innate training.
We consider the following $N$-dimensional Leaky ESN setups on time steps $T=\{\cdots, -2, -1, 0, 1, 2, \cdots\}$;
$$
\def\bm#1{{\boldsymbol #1}}
\begin{align*}\\
\bm{x}(t+1) &= (1-a)\bm{x}(t)+a\tanh\left(g W^\text{net} \bm{x}(t) + f^\text{in}(t)\right),\\
\\\end{align*}$$
where $f^\text{in}: T \to \mathbb{R}^N$ is a input time series. Originally in the innate training, a pulse-like function is used as $f^\text{in}$ causing temporary synchronization. You can use any sort of input function as long as it satisfies this property like in [Goudar, V., & Buonomano, D. V. (2018)](https://elifesciences.org/articles/31134) using **auditory data** or [Inoue, K., *et al.,* (2020)](https://www.science.org/doi/10.1126/sciadv.abb3989) using **relaxation process** of the echo state network. The cost function in innate training can be represented as follows:
$$
\def\bm#1{{\boldsymbol #1}}
\begin{align*}\\
\mathcal{L}^\text{innate}_{W^\text{net}} = \sum_{t=t_0}^{t_1} \| \hat{\bm{x}}(t) - \bm{x}(t) \|^2
\\\end{align*}$$
where $\hat{\bm{x}}(t)$ is the innate trajectory started by a randomly-chosen initial condition $\hat{\bm{x}}(t_0)$ and sampled the initial internal conncetion $W^\text{net}$.

First, let's fill in the blank in the following `emulate` to support this process.

Specification of `emulate`:
- Arguments:
    - `ts` (`list` or `np.ndarray` or `range(...)`): List of sampled time (*e. g.*, $[0, 1, 2, ..., T-1]$)
    - `net` (`InnateESN`): An instance for ESN ($\tanh, W^\text{net}$)
    - `f_in` (`Callable`): An instance of $f^\text{in}(t)$
    - `innate_range` (`tuple(int, int)`): Time range of pre-training (*i.e.* training phase, $[t_0, t_1)$)
    - `innate_neuron` (`list, slice, optional`): Tuned nodes.
    - `innate_every` (`int`): Frequency of `InnateESN.train` call.
    - `innate_func` (`Callable`): An instance of $\hat{\bm{x}}: T \to \mathbb{R}^{\cdots \times N}$
- Return
    - `np.ndarray` or `None`: Return network dynamics if `record=True`. Otherwise, return nothing.


<details>
<summary>
Hints
</summary>

- Call `net.train` we have implemented.
- Do not forget to set the `neuron_list` option.

In [ ]:
%%run_and_save ./answer/innate_emulate.py

def emulate(
        ts, net, f_in=None, record=True, 
        innate_range=None, innate_func=None,
        innate_neuron=None, innate_every=2, prefix=""):
    if record:
        rec_net = np.zeros((len(ts), *net.x.shape))

    pbar = tqdm(ts, leave=False)
    for cnt, t in enumerate(pbar):
        pbar.set_description("{}t={:.0f}".format(prefix, t))
        # step function
        u_in = np.zeros_like(net.x)
        if f_in is not None:
            u_in += f_in(t)
        net.step(u_in)
        # recording
        if record:
            rec_net[cnt] = net.x
        # innate learning 
        if (innate_range is not None) and (innate_range[0] <= t < innate_range[1]):
            if cnt % innate_every == 0:
                x_target = ... # TODO use `innate_func`.
                net.train(x_target, neuron_list=innate_neuron)  #RIGHT use `net.train` and specify `neuron_list=innate_neuron`.
    pbar.close()
    if record:
        return ts, rec_net
    

The following functions depend on `emulate`.

Run the following cells.

In [ ]:
def create_target(record_range, net, f_in, rnd=None):
    rnd = rnd if rnd else np.random
    net.reset(rnd.uniform(low=-1, high=1, size=(f_in.dim, net.dim)))
    rec_t, rec_net = emulate(
        range(*record_range), net, f_in=f_in, prefix="sample ")
    return (rec_t, rec_net)


def train_network(record_range, net, f_in, rec_target, 
                  innate_range, innate_neuron, innate_every, rnd=None):
    rnd = rnd if rnd else np.random
    innate_func = interp1d(rec_target[1], x=rec_target[0], kind=1, axis=0)
    net.reset(rnd.uniform(low=-1, high=1, size=(f_in.dim, net.dim)))
    rec_t, rec_net = emulate(
        range(*record_range), net, f_in=f_in, prefix="train ",
        innate_func=innate_func, innate_neuron=innate_neuron,
        innate_range=innate_range, innate_every=innate_every)
    return (rec_t, rec_net)


def eval_network(record_range, net, f_in, eval_num=5, rnd=None):
    rnd = rnd if rnd else np.random
    net.reset(rnd.uniform(low=-1, high=1, size=(eval_num, f_in.dim, net.dim)))
    rec_t, rec_net = emulate(
        range(*record_range), net, f_in=f_in, prefix="eval ")
    return (rec_t, rec_net)


def eval_error(rec_eval, rec_target, innate_range, innate_neuron):
    begin_id = innate_range[0] - record_range[0]
    end_id = innate_range[1] - record_range[0]
    diff = rec_eval[1] - rec_target[1][:, None]  # -> [Time_steps, Eval_num, W_in_num, Net_dim]
    norm = (diff[begin_id:end_id, ..., innate_neuron] ** 2).sum(axis=-1)  # -> [T, E, W]
    mse = norm.mean(axis=0)  # -> [E, W]

    var = (rec_target[1][begin_id:end_id, ..., innate_neuron]**2).sum(axis=(0, 2)) # -> [W]
    nmse = norm.sum(axis=0) / var  # -> [E, W]
    return {"mse": mse, "nmse": nmse}

# 5. Demonstration

Now, let's pre-train the ESN's internal connection. 

Please change the following parameters freely to understand the behavior.

- Global parameters
    - `seed` (`int`): Random seed.
    - `w_in_num` (`int`): The number of $\bm{f}^\text{in}$. Default to 1 but you can freely increase it.
    - `save_dir` (`save`): Output directory. Default to `./output`.
- ESN's parameters
    - `dim` (`int`): Dimension.
    - `a` (`float`): Leaky rate.
    - `g` (`float`): Spectral radius.
    - `p` (`float`): Connection density.
- Pulse-like function ($f^\text{in}$)'s parameters
    - `pulse_amp` (`float`): Pulse amplitude.
    - `pulse_period` (`float`): Pulse width.
- Pre-training parameters
    - `alpha` (`float`): InnateESN's parameter.
    - `innate_epoch` (`int`): The number of training epochs.
    - `innate_period` (`int`): Tune period ($|t_1 - t_0|$).
    - `innate_rate` (`int`): The ratio of tuned nodes.
    - `innate_every` (`int`): Frequency of weight update.
    - `washout_period` (`int`): Washout periods to discard the transient dynamics.
    - `record_period` (`int`): Emulation periods.

In [ ]:
# You can change these parameters freely

seed = 1234
w_in_num = 1
save_dir = "./output"
rnd = np.random.RandomState(seed)

dim, a, g, p = 1000, 0.1, 1.6, 0.25

pulse_amp, pulse_period = 10.0, 50

alpha = 1.0
innate_epoch = 30
innate_period, innate_rate, innate_every = 2000, 0.1, 4
washout_period, record_period = 1000, 5000

os.makedirs(save_dir, exist_ok=True)

with open(f"{save_dir}/params.json", mode="w") as f:
    json.dump({
        "pulse_amp": pulse_amp,
        "pulse_period": pulse_period,
        "innate_period": innate_period,
        "innate_rate": innate_rate,
        "innate_every": innate_every
    }, f, indent=4)

Next, let's create the instances.

Please understand the process in `f_in`. 

All training results will be stored on `save_dir`, which is reused in the next notebook `innate_training_post.ipynb`.

In [ ]:
net = InnateESN(dim, a=a, g=g, p=p, alpha=alpha, rnd=rnd)
w_in = rnd.uniform(size=(w_in_num, dim), low=-1.0, high=1.0)

innate_range = [0, innate_period]
innate_neuron = list(range(0, int(innate_rate * net.dim)))
record_range = [-washout_period, record_period]
pulse_range = [-pulse_period, 0]

plot_range = list(range(5))

def f_in(t):
    if -pulse_period <= t < 0:
        return pulse_amp * w_in[:w_in_num]
    else:
        return 0.0

f_in.dim = w_in_num

os.makedirs(save_dir, exist_ok=True)
np.save(f"{save_dir}/w_in.npy", w_in)
net.to_pickle(f"{save_dir}/net_init.pkl")

Here, innate trajectory $\hat{\bm{x}}(t)$ is created.

You will see the complicated trajectory.

In [ ]:
rec_target = create_target(record_range, net, f_in, rnd=rnd)
fig_target = show_trajectory(
    *rec_target, plot_range, lw=1.0, color="red",
    title="innate trajectory",
    pulse_range=pulse_range, innate_range=innate_range)
np.savez_compressed(f"{save_dir}/rec_target.npz", *rec_target)
fig_target.savefig(f"{save_dir}/rec_target.png", dpi=200)

The gray area represents a time period when pulse input is given.

The pink area represents the pre-training period $[t_0, t_1)$.

You will see the system fails to reconstruct the innate trajectory (dot lines) in the next figure.

In [ ]:
rec_init = eval_network(record_range, net, f_in, rnd=rnd)
fig = show_trajectory(
    *rec_target, plot_range, lw=1.0, ls=":", 
    clear=True, color="red", pulse_range=pulse_range, innate_range=innate_range)
fig = show_trajectory(
    *rec_init, plot_range, lw=0.5, ls="-", fig=fig,
    cmap=plt.get_cmap("tab10"), title="eval (before training)")
fig.savefig(f"{save_dir}/eval/init.png", dpi=200)

Now, it's time to start pre-training.

It will take around 20~30 mins. Please be patient until it finishes.

In [ ]:
from ipywidgets import Output
from IPython.display import display, clear_output

record = {}
best_score = np.array(np.inf)
figs = defaultdict(lambda: None)

pbar = trange(innate_epoch)
out_tqdm = Output()
out_figure = Output()
display(out_tqdm)
display(out_figure)
for epoch in pbar:
    pbar.set_description("best:{:.2e}".format(best_score))
    with out_tqdm:
        clear_output(wait=True)
        # training
        rec_train = train_network(
            record_range, net, f_in, rec_target, 
            innate_range, innate_neuron, innate_every, rnd=rnd)
        figs["train"] = show_trajectory(
            *rec_target, plot_range, lw=1.0, ls=":", 
            clear=True, color="red", fig=figs["train"],
            pulse_range=pulse_range, innate_range=innate_range)
        figs["train"] = show_trajectory(
            *rec_train, plot_range, lw=1.0, ls="-", 
            fig=figs["train"], cmap=plt.get_cmap("tab10"), title=f"train (epoch #{epoch})")
        figs["train"].savefig("{}/train/{:03d}.png".format(save_dir, epoch), dpi=200)

        # evaluation
        rec_eval = eval_network(record_range, net, f_in, rnd=rnd)
        figs["eval"] = show_trajectory(
            *rec_target, plot_range, lw=1.0, ls=":", 
            clear=True, color="red", fig=figs["eval"],
            pulse_range=pulse_range, innate_range=innate_range)
        figs["eval"] = show_trajectory(
            *rec_eval, plot_range, lw=0.5, ls="-", 
            fig=figs["eval"], cmap=plt.get_cmap("tab10"), title=f"eval (epoch #{epoch})")
        figs["eval"].savefig("{}/eval/{:03d}.png".format(save_dir, epoch), dpi=200)

        # calc. error
        rec = eval_error(rec_eval, rec_target, innate_range, innate_neuron)
        best_score = min(best_score, rec["mse"].sum())
        rec["best"] = best_score

        for key, val in rec.items():
            if not (key in record):
                record[key] = np.zeros((innate_epoch, *val.shape))
            record[key][epoch] = val

    with out_figure:
        clear_output(wait=True)
        figs["mse"] = show_error(record["mse"][:epoch + 1], fig=figs["mse"])
        figs["mse"].savefig(f"{save_dir}/mse.png", dpi=200)
        for _name, fig in figs.items():
            size = fig._fig.get_size_inches()
            fig._fig.set_size_inches(8, 3)
            display(fig._fig)
            fig._fig.set_size_inches(size)

net.to_pickle(f"{save_dir}/net_term.pkl")
np.savez_compressed(f"{save_dir}/record.npz", **record)

for _name, fig in figs.items():
    fig.close()

Can you see the network behavior stably reproduce complicated high-dimensional chaos?

Let's exploit it in next notebook.

## Advanced
- What is the node dependency?
- What does occur with a long innate trajectory?
- Let's increase `w_in_dim` and check what happens.